<a href="https://colab.research.google.com/github/rick1270/DS-Unit-1-Sprint-4-Statistical-Tests-and-Experiments/blob/master/DS_Unit_1_Sprint_Challenge_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Science Unit 1 Sprint Challenge 4

## Exploring Data, Testing Hypotheses

In this sprint challenge you will look at a dataset of people being approved or rejected for credit.

https://archive.ics.uci.edu/ml/datasets/Credit+Approval

Data Set Information: This file concerns credit card applications. All attribute names and values have been changed to meaningless symbols to protect confidentiality of the data. This dataset is interesting because there is a good mix of attributes -- continuous, nominal with small numbers of values, and nominal with larger numbers of values. There are also a few missing values.

Attribute Information:
- A1: b, a.
- A2: continuous.
- A3: continuous.
- A4: u, y, l, t.
- A5: g, p, gg.
- A6: c, d, cc, i, j, k, m, r, q, w, x, e, aa, ff.
- A7: v, h, bb, j, n, z, dd, ff, o.
- A8: continuous.
- A9: t, f.
- A10: t, f.
- A11: continuous.
- A12: t, f.
- A13: g, p, s.
- A14: continuous.
- A15: continuous.
- A16: +,- (class attribute)

Yes, most of that doesn't mean anything. A16 (the class attribute) is the most interesting, as it separates the 307 approved cases from the 383 rejected cases. The remaining variables have been obfuscated for privacy - a challenge you may have to deal with in your data science career.

Sprint challenges are evaluated based on satisfactory completion of each part. It is suggested you work through it in order, getting each aspect reasonably working, before trying to deeply explore, iterate, or refine any given step. Once you get to the end, if you want to go back and improve things, go for it!

## Part 1 - Load and validate the data

- Load the data as a `pandas` data frame.
- Validate that it has the appropriate number of observations (you can check the raw file, and also read the dataset description from UCI).
- UCI says there should be missing data - check, and if necessary change the data so pandas recognizes it as na
- Make sure that the loaded features are of the types described above (continuous values should be treated as float), and correct as necessary

This is review, but skills that you'll use at the start of any data exploration. Further, you may have to do some investigation to figure out which file to load from - that is part of the puzzle.

In [0]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
credit_URL = 'https://archive.ics.uci.edu/ml/machine-learning-databases/credit-screening/crx.data'
credit_data = pd.read_csv(credit_URL, header = None, names = ['A1', 'A2', 'A3','A4', 'A5', 'A6', 'A7', 'A8', 'A9', 'A10', 'A11', 'A12', 'A13', 'A14', 'A15', 'A16'])
credit_data.replace('?', np.nan, inplace = True)
credit_data.replace(['f', '-', 'a', 't', '+', 'b'], [0, 0, 0, 1, 1, 1], inplace = True)
credit_data.head()



,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16
0,1.0,30.83,0.000,u,g,w,v,1.25,1,1,1,0,g,00202,0,1
1,0.0,58.67,4.460,u,g,q,h,3.04,1,1,6,0,g,00043,560,1
2,0.0,24.50,0.500,u,g,q,h,1.50,1,0,0,0,g,00280,824,1
3,1.0,27.83,1.540,u,g,w,v,3.75,1,1,5,1,g,00100,3,1
4,1.0,20.17,5.625,u,g,w,v,1.71,1,0,0,0,s,00120,0,1


In [0]:
credit_data.describe(include ='all')

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16
count,678.000000,678.000000,690.000000,684,684,681,681,690.000000,690.000000,690.000000,690.00000,690.000000,690,677.000000,690.000000,690.000000
unique,NaN,NaN,NaN,3,3,14,9,NaN,NaN,NaN,NaN,NaN,3,NaN,NaN,NaN
top,NaN,NaN,NaN,u,g,c,v,NaN,NaN,NaN,NaN,NaN,g,NaN,NaN,NaN
freq,NaN,NaN,NaN,519,519,137,399,NaN,NaN,NaN,NaN,NaN,625,NaN,NaN,NaN
mean,0.690265,31.568171,4.758725,NaN,NaN,NaN,NaN,2.223406,0.523188,0.427536,2.40000,0.457971,NaN,184.014771,1017.385507,0.444928
std,0.462725,11.957862,4.978163,NaN,NaN,NaN,NaN,3.346513,0.499824,0.495080,4.86294,0.498592,NaN,173.806768,5210.102598,0.497318
min,0.000000,13.750000,0.000000,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.00000,0.000000,NaN,0.000000,0.000000,0.000000
25%,0.000000,22.602500,1.000000,NaN,NaN,NaN,NaN,0.165000,0.000000,0.000000,0.00000,0.000000,NaN,75.000000,0.000000,0.000000
50%,1.000000,28.460000,2.750000,NaN,NaN,NaN,NaN,1.000000,1.000000,0.000000,0.00000,0.000000,NaN,160.000000,5.000000,0.000000
75%,1.000000,38.230000,7.207500,NaN,NaN,NaN,NaN,2.625000,1.000000,1.000000,3.00000,1.000000,NaN,276.000000,395.500000,1.000000


In [0]:
credit_data.isna().sum()

0     12
1     12
2      0
3      6
4      6
5      9
6      9
7      0
8      0
9      0
10     0
11     0
12     0
13    13
14     0
15     0
dtype: int64

In [0]:
credit_data[["A2", "A9", "A14"]] = credit_data[["A2", "A9", "A14"]].apply(pd.to_numeric)
credit_data.dtypes

#A1: b, a.
#A2: continuous.
#A3: continuous.
#A4: u, y, l, t.
#A5: g, p, gg.
#A6: c, d, cc, i, j, k, m, r, q, w, x, e, aa, ff.
#A7: v, h, bb, j, n, z, dd, ff, o.
#A8: continuous.
#A9: t, f.
#A10: t, f.
#A11: continuous.
#A12: t, f.
#A13: g, p, s.
#A14: continuous.
#A15: continuous.
#A16: +,- (class attribute)

A1     float64
A2     float64
A3     float64
A4      object
A5      object
A6      object
A7      object
A8     float64
A9       int64
A10      int64
A11      int64
A12      int64
A13     object
A14    float64
A15      int64
A16      int64
dtype: object

## Part 2 - Exploring data, Testing hypotheses

The only thing we really know about this data is that A16 is the class label. Besides that, we have 6 continuous (float) features and 9 categorical features.

Explore the data: you can use whatever approach (tables, utility functions, visualizations) to get an impression of the distributions and relationships of the variables. In general, your goal is to understand how the features are different when grouped by the two class labels (`+` and `-`).

For the 6 continuous features, how are they different when split between the two class labels? Choose two features to run t-tests (again split by class label) - specifically, select one feature that is *extremely* different between the classes, and another feature that is notably less different (though perhaps still "statistically significantly" different). You may have to explore more than two features to do this.

For the categorical features, explore by creating "cross tabs" between them and the class label, and apply the Chi-squared test to them. There are 9 categorical features - as with the t-test, try to find one where the Chi-squared test returns an extreme result (rejecting the null that the data are independent), and one where it is less extreme.

**NOTE** - "less extreme" just means smaller test statistic/larger p-value. Even the least extreme differences may be strongly statistically significant.

Your *main* goal is the hypothesis tests, so don't spend too much time on the exploration/visualization piece. That is just a means to an end. This is challenging, so manage your time and aim for a baseline of at least running two t-tests and two Chi-squared tests before polishing. And don't forget to answer the questions in part 3, even if your results in this part aren't what you want them to be.

In [0]:
class_0 = credit_data[credit_data['A16'] == 0]
class_0 ['A4'].sample(10)

312    u
287    u
619    y
305    y
630    u
90     y
674    u
296    u
377    y
311    y
Name: A4, dtype: object

In [0]:
class_1 = credit_data[credit_data['A16'] == 1]
class_1.sample(10)

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16
564,1.0,42.17,5.040,u,g,q,h,12.750,1,0,0,1,g,92.0,0,1
186,1.0,40.00,6.500,u,g,aa,bb,3.500,1,1,1,0,g,0.0,500,1
589,1.0,25.33,0.580,u,g,c,v,0.290,1,1,7,1,g,96.0,5124,1
496,0.0,25.00,0.875,u,g,x,h,1.040,1,0,0,1,g,160.0,5860,1
571,1.0,21.00,3.000,y,p,d,v,1.085,1,1,8,1,g,160.0,1,1
199,1.0,22.58,10.040,u,g,x,v,0.040,1,1,9,0,g,60.0,396,1
22,0.0,47.75,8.000,u,g,c,v,7.875,1,1,6,1,g,0.0,1260,1
64,1.0,26.67,4.250,u,g,cc,v,4.290,1,1,1,1,g,120.0,0,1
33,0.0,36.75,5.125,u,g,e,v,5.000,1,0,0,1,g,0.0,4000,1
26,0.0,47.00,13.000,u,g,i,bb,5.165,1,1,9,1,g,0.0,0,1


In [0]:
class_0.describe(include = 'all')

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16
count,374.000000,373.000000,383.000000,381,381,378,378,383.000000,383.000000,383.000000,383.000000,383.000000,383,376.000000,383.000000,383.0
unique,NaN,NaN,NaN,2,2,14,9,NaN,NaN,NaN,NaN,NaN,3,NaN,NaN,NaN
top,NaN,NaN,NaN,u,g,c,v,NaN,NaN,NaN,NaN,NaN,g,NaN,NaN,NaN
freq,NaN,NaN,NaN,263,263,75,230,NaN,NaN,NaN,NaN,NaN,338,NaN,NaN,NaN
mean,0.700535,29.808231,3.839948,NaN,NaN,NaN,NaN,1.257924,0.201044,0.224543,0.631854,0.443864,NaN,199.699468,198.605744,0.0
std,0.458637,10.919291,4.337662,NaN,NaN,NaN,NaN,2.120481,0.401305,0.417827,1.900049,0.497489,NaN,181.564835,671.608839,0.0
min,0.000000,15.170000,0.000000,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,0.000000,0.000000,0.0
25%,0.000000,22.000000,0.835000,NaN,NaN,NaN,NaN,0.125000,0.000000,0.000000,0.000000,0.000000,NaN,100.000000,0.000000,0.0
50%,1.000000,27.330000,2.210000,NaN,NaN,NaN,NaN,0.415000,0.000000,0.000000,0.000000,0.000000,NaN,167.500000,1.000000,0.0
75%,1.000000,34.830000,5.000000,NaN,NaN,NaN,NaN,1.500000,0.000000,0.000000,0.000000,1.000000,NaN,272.000000,67.000000,0.0


In [0]:
class_0.mean()

A1       0.700535
A2      29.808231
A3       3.839948
A8       1.257924
A9       0.201044
A10      0.224543
A11      0.631854
A12      0.443864
A14    199.699468
A15    198.605744
A16      0.000000
dtype: float64

In [0]:
class_1.mean()

A1        0.677632
A2       33.720492
A3        5.904951
A8        3.427899
A9        0.925081
A10       0.680782
A11       4.605863
A12       0.475570
A14     164.421927
A15    2038.859935
A16       1.000000
dtype: float64

In [94]:
credit_tests ={}
cat = ['A1', 'A2', 'A3', 'A8', 'A9', 'A10', 'A11', 'A12', 'A14', 'A15', 'A16']
for i in range (1, len(cat)):
  credit_tests[cat[i]] = stats.ttest_ind(class_0[cat[i]], class_1[cat[i]],nan_policy='omit')
  

  
for credit in credit_tests.keys():
  print (credit, credit_tests[credit])

A2 Ttest_indResult(statistic=-4.2922156166315535, pvalue=2.0276637071781407e-05)
A3 Ttest_indResult(statistic=-5.52998337614816, pvalue=4.551680702308068e-08)
A8 Ttest_indResult(statistic=-8.935819983773698, pvalue=3.6710537401601785e-18)
A9 Ttest_indResult(statistic=-27.245398535467622, pvalue=1.9368208799738917e-111)
A10 Ttest_indResult(statistic=-13.52517266303669, pvalue=3.941258302842775e-37)
A11 Ttest_indResult(statistic=-11.667004222431277, pvalue=7.957718568079967e-29)
A12 Ttest_indResult(statistic=-0.8299262276960956, pvalue=0.4068683677059408)
A14 Ttest_indResult(statistic=2.6358251986645476, pvalue=0.008586135473979569)
A15 Ttest_indResult(statistic=-4.680216020964486, pvalue=3.4520256956287944e-06)
A16 Ttest_indResult(statistic=-inf, pvalue=0.0)


In [0]:
pd.crosstab(class_0 ['A4'], class_1 ['A4'])

""


## Part 3 - Analysis and Interpretation

Now that you've looked at the data, answer the following questions:

- Interpret and explain the two t-tests you ran - what do they tell you about the relationships between the continuous features you selected and the class labels?
- Interpret and explain the two Chi-squared tests you ran - what do they tell you about the relationships between the categorical features you selected and the class labels?
- What was the most challenging part of this sprint challenge?

Answer with text, but feel free to intersperse example code/results or refer to it from earlier.

Regarding the continuous features, A12 appears to be very different between the two groups (approved, not approved) but the P Value indicates no statistical significance.  Conversely,  A1 appear to be very similar to the null hypothesis, however, the P Value indicates statistical significance.

I ran out of time and was unable to complete the Chi-Squared test on the categorical values

Getting crosstabs to work

